<a href="https://colab.research.google.com/github/WPNCH/MachineLearningPlayGround/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving XAUUSD_2021_M1 - Clean.xlsx to XAUUSD_2021_M1 - Clean.xlsx


In [ ]:
import io
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data_utils
from torch.utils.data import Dataset
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import DataLoader
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
df_original = pd.read_excel('XAUUSD_2021_M1 - Clean.xlsx')

In [ ]:
df_new = df_original.fillna(0)

In [ ]:
df_new['7MA'] = df_new['Close'].rolling(7).mean()

In [ ]:
df_new['14MA'] = df_new['Close'].rolling(14).mean()

In [ ]:
df_new = df_new.fillna(0)

In [ ]:
df_new = df_new.loc[df_new['14RSI']>0]

In [ ]:
df_new['Overbought'] = np.where(df_new['14RSI']>=70,1,0)

In [ ]:
df_new['Oversold'] = np.where(df_new['14RSI']<=30,1,0)

In [ ]:
df_new['DayOfWeek'] = pd.to_datetime(df_new['DateTime']).dt.dayofweek

In [ ]:
df_new_1 = df_new.copy()

In [ ]:
target_close = "Close"


forecast_lead = 15
target = f"{target_close}_lead{forecast_lead}"



In [ ]:
df_new_1[target] = df_new_1[target_close].shift(-forecast_lead)


In [ ]:
df_new_1 = df_new_1.iloc[:-forecast_lead]

In [ ]:
df_new_1

,Index,DateTime,Open,High,Low,Close,14RSI,7MA,14MA,Overbought,Oversold,DayOfWeek,Close_lead15
14,15,2021-01-03 18:14:00,1908.818,1909.405,1908.695,1908.805,44.324609,1908.286286,1908.648714,0,0,6,1910.765
15,16,2021-01-03 18:15:00,1908.785,1908.795,1907.978,1908.295,44.035019,1908.345857,1908.625643,0,0,6,1911.158
16,17,2021-01-03 18:16:00,1908.295,1908.688,1908.158,1908.688,47.713436,1908.363000,1908.514929,0,0,6,1912.568
17,18,2021-01-03 18:17:00,1908.695,1909.558,1908.578,1909.558,36.720356,1908.575857,1908.468500,0,0,6,1912.845
18,19,2021-01-03 18:18:00,1909.608,1909.678,1909.378,1909.668,45.131815,1908.837714,1908.526929,0,0,6,1913.718
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85356,85357,2021-03-30 23:41:00,1679.198,1679.198,1678.878,1678.988,44.052863,1679.018000,1679.188000,0,0,1,1679.335
85357,85358,2021-03-30 23:42:00,1678.988,1679.168,1678.911,1679.108,36.796537,1679.028000,1679.156571,0,0,1,1679.355
85358,85359,2021-03-30 23:43:00,1679.088,1679.658,1679.038,1679.545,40.756303,1679.143286,1679.144929,0,0,1,1679.398
85359,85360,2021-03-30 23:44:00,1679.545,1679.715,1679.445,1679.715,46.932631,1679.292857,1679.168286,0,0,1,1679.385


In [ ]:
df_shifted = df_new_1.copy()

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc_df_ob = pd.DataFrame(enc.fit_transform(df_shifted[['Overbought']]).toarray())
enc_df_os = pd.DataFrame(enc.fit_transform(df_shifted[['Oversold']]).toarray())
enc_df_dow = pd.DataFrame(enc.fit_transform(df_shifted[['DayOfWeek']]).toarray())

In [ ]:
enc_df_ob = enc_df_ob.rename(columns={0:'OBFalse',1:'OBTrue'})
enc_df_os = enc_df_os.rename(columns={0:'OSFalse',1:'OSTrue'})
enc_df_dow = enc_df_dow.rename(columns={0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat'})

In [ ]:
enc_df_dow

,Mon,Tue,Wed,Thu,Fri,Sat
0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
85337,0.0,1.0,0.0,0.0,0.0,0.0
85338,0.0,1.0,0.0,0.0,0.0,0.0
85339,0.0,1.0,0.0,0.0,0.0,0.0
85340,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
df_new_merge = df_shifted.copy().drop('Index',axis = 1)

In [ ]:
df_new_merge = df_new_merge.reset_index()


In [ ]:
df_new_merge = df_new_merge.drop('index',axis=1)

In [ ]:
df_new_merge = df_new_merge.join(enc_df_ob)
df_new_merge = df_new_merge.join(enc_df_os)
df_new_merge = df_new_merge.join(enc_df_dow)

In [ ]:
df_new_merge

,DateTime,Open,High,Low,Close,14RSI,7MA,14MA,Overbought,Oversold,...,OBFalse,OBTrue,OSFalse,OSTrue,Mon,Tue,Wed,Thu,Fri,Sat
0,2021-01-03 18:14:00,1908.818,1909.405,1908.695,1908.805,44.324609,1908.286286,1908.648714,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2021-01-03 18:15:00,1908.785,1908.795,1907.978,1908.295,44.035019,1908.345857,1908.625643,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2021-01-03 18:16:00,1908.295,1908.688,1908.158,1908.688,47.713436,1908.363000,1908.514929,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2021-01-03 18:17:00,1908.695,1909.558,1908.578,1909.558,36.720356,1908.575857,1908.468500,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2021-01-03 18:18:00,1909.608,1909.678,1909.378,1909.668,45.131815,1908.837714,1908.526929,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85337,2021-03-30 23:41:00,1679.198,1679.198,1678.878,1678.988,44.052863,1679.018000,1679.188000,0,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
85338,2021-03-30 23:42:00,1678.988,1679.168,1678.911,1679.108,36.796537,1679.028000,1679.156571,0,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
85339,2021-03-30 23:43:00,1679.088,1679.658,1679.038,1679.545,40.756303,1679.143286,1679.144929,0,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
85340,2021-03-30 23:44:00,1679.545,1679.715,1679.445,1679.715,46.932631,1679.292857,1679.168286,0,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
df_new_merge.columns.values

array(['DateTime', 'Open', 'High', 'Low', 'Close', '14RSI', '7MA', '14MA',
       'Overbought', 'Oversold', 'NewClose', 'Date', 'Time', 'DayOfWeek',
       'OBFalse', 'OBTrue', 'OSFalse', 'OSTrue', 'Mon', 'Tue', 'Wed',
       'Thu', 'Fri', 'Sat'], dtype=object)

In [ ]:
df_final = df_new_merge.copy().drop(['DayOfWeek','Overbought','Oversold'],axis=1).set_index('DateTime')

In [ ]:
df_final

,Open,High,Low,Close,14RSI,7MA,14MA,Close_lead15,OBFalse,OBTrue,OSFalse,OSTrue,Mon,Tue,Wed,Thu,Fri,Sat
DateTime,,,,,,,,,,,,,,,,,,
2021-01-03 18:14:00,1908.818,1909.405,1908.695,1908.805,44.324609,1908.286286,1908.648714,1910.765,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2021-01-03 18:15:00,1908.785,1908.795,1907.978,1908.295,44.035019,1908.345857,1908.625643,1911.158,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2021-01-03 18:16:00,1908.295,1908.688,1908.158,1908.688,47.713436,1908.363000,1908.514929,1912.568,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2021-01-03 18:17:00,1908.695,1909.558,1908.578,1909.558,36.720356,1908.575857,1908.468500,1912.845,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2021-01-03 18:18:00,1909.608,1909.678,1909.378,1909.668,45.131815,1908.837714,1908.526929,1913.718,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:41:00,1679.198,1679.198,1678.878,1678.988,44.052863,1679.018000,1679.188000,1679.335,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2021-03-30 23:42:00,1678.988,1679.168,1678.911,1679.108,36.796537,1679.028000,1679.156571,1679.355,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2021-03-30 23:43:00,1679.088,1679.658,1679.038,1679.545,40.756303,1679.143286,1679.144929,1679.398,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
UsingData = df_final.iloc[:40000,:].copy()

In [ ]:
UsingData

,Open,High,Low,Close,14RSI,7MA,14MA,Close_lead15,OBFalse,OBTrue,OSFalse,OSTrue,Mon,Tue,Wed,Thu,Fri,Sat
DateTime,,,,,,,,,,,,,,,,,,
2021-01-03 18:14:00,1908.818,1909.405,1908.695,1908.805,44.324609,1908.286286,1908.648714,1910.765,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2021-01-03 18:15:00,1908.785,1908.795,1907.978,1908.295,44.035019,1908.345857,1908.625643,1911.158,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2021-01-03 18:16:00,1908.295,1908.688,1908.158,1908.688,47.713436,1908.363000,1908.514929,1912.568,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2021-01-03 18:17:00,1908.695,1909.558,1908.578,1909.558,36.720356,1908.575857,1908.468500,1912.845,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2021-01-03 18:18:00,1909.608,1909.678,1909.378,1909.668,45.131815,1908.837714,1908.526929,1913.718,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-11 21:50:00,1820.485,1820.655,1820.465,1820.655,39.186100,1820.679286,1820.589500,1820.005,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2021-02-11 21:51:00,1820.665,1820.695,1820.415,1820.415,49.927780,1820.629286,1820.588571,1820.195,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2021-02-11 21:52:00,1820.445,1820.525,1819.652,1819.692,49.688548,1820.463143,1820.518143,1820.255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
test_start = '2021-02-01'
df_train = UsingData.loc[:test_start].copy()
df_test = UsingData.loc[test_start:].copy()

In [ ]:
df_train_scaled = df_train.copy()
df_test_scaled = df_test.copy()

col_names = ['Open', 'High','Low','14RSI','7MA','14MA','Close','Close_lead15']
train_features = df_train_scaled[col_names]
test_features = df_test_scaled[col_names]

# Use scaler of choice; here Standard scaler is used
scaler = StandardScaler().fit(train_features.values)
train_features = scaler.fit_transform(train_features.values)
test_features = scaler.transform(test_features.values)

df_train_scaled[col_names] = train_features
df_test_scaled[col_names] = test_features

In [ ]:
df_test_scaled

,Open,High,Low,Close,14RSI,7MA,14MA,Close_lead15,OBFalse,OBTrue,OSFalse,OSTrue,Mon,Tue,Wed,Thu,Fri,Sat,Model forecast
DateTime,,,,,,,,,,,,,,,,,,,
2021-02-01 00:00:00,-0.212223,-0.211324,-0.212743,-0.218634,-0.624288,-0.202040,-0.198128,-0.196162,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.223317
2021-02-01 00:01:00,-0.219037,-0.214223,-0.213236,-0.204136,-1.193935,-0.203925,-0.198828,-0.184530,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.223321
2021-02-01 00:02:00,-0.205612,-0.202917,-0.195255,-0.198916,-0.372962,-0.204075,-0.198822,-0.190331,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.222828
2021-02-01 00:03:00,-0.199609,-0.206686,-0.200272,-0.210254,-0.024252,-0.205433,-0.199549,-0.193812,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.216619
2021-02-01 00:04:00,-0.210280,-0.211237,-0.207349,-0.204049,-0.347526,-0.207720,-0.200315,-0.205415,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.216768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-11 21:50:00,-1.365787,-1.370573,-1.356210,-1.360745,-0.709419,-1.360338,-1.363248,-1.379245,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.149675
2021-02-11 21:51:00,-1.360568,-1.369413,-1.357660,-1.367705,-0.031857,-1.361788,-1.363275,-1.373733,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.144791
2021-02-11 21:52:00,-1.366947,-1.374341,-1.379789,-1.388670,-0.046947,-1.366606,-1.365318,-1.371993,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.140821


In [ ]:
features = list(df_final.columns.difference(['Close']))

In [ ]:
target

'Close_lead15'

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [ ]:
i = 27
sequence_length = 4

train_dataset = SequenceDataset(
    df_train_scaled,
    target=target,
    features=features,
    sequence_length=sequence_length
)

X, y = train_dataset[i]
print(X)

tensor([[1.2906, 1.8225, 1.3373, 1.3171, 0.0000, 1.3334, 1.3323, 0.0000, 0.0000,
         1.0000, 1.0000, 0.0000, 1.3260, 1.0000, 0.0000, 0.0000, 0.0000],
        [1.3000, 2.0789, 1.3425, 1.2991, 0.0000, 1.3406, 1.3526, 0.0000, 0.0000,
         1.0000, 1.0000, 0.0000, 1.3431, 1.0000, 0.0000, 0.0000, 0.0000],
        [1.3099, 2.0324, 1.3448, 1.2184, 0.0000, 1.3448, 1.3558, 0.0000, 0.0000,
         1.0000, 1.0000, 0.0000, 1.3486, 1.0000, 0.0000, 0.0000, 0.0000],
        [1.3200, 2.0938, 1.3448, 1.2259, 0.0000, 1.3575, 1.3587, 0.0000, 0.0000,
         1.0000, 1.0000, 0.0000, 1.3533, 1.0000, 0.0000, 0.0000, 0.0000]])


In [ ]:
torch.manual_seed(101)

batch_size = 4
sequence_length = 30

train_dataset = SequenceDataset(
    df_train_scaled,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test_scaled,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([4, 30, 17])
Target shape: torch.Size([4])


In [ ]:
class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_feat, hidden_units):
        super().__init__()
        self.num_feat = num_feat  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_feat,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()

        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [ ]:
learning_rate = 5e-5
num_hidden_units = 16

model = ShallowRegressionLSTM(num_feat=len(features), hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()

    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):

    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")


print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

for ix_epoch in range(2):
    print(f"Epoch {ix_epoch}\n---------")
    train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_model(test_loader, model, loss_function)
    print()

Untrained test
--------
Test loss: 1.090001096854472

Epoch 0
---------
Train loss: 0.13542542901140542
Test loss: 0.14578582934094894

Epoch 1
---------
Train loss: 0.0045288299957675065
Test loss: 0.0872060327573098



In [ ]:
def predict(data_loader, model):

    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)

    return output


train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
df_train_scaled[ystar_col] = predict(train_eval_loader, model).numpy()
df_test_scaled[ystar_col] = predict(test_loader, model).numpy()


In [ ]:
df_out = pd.concat((df_train_scaled, df_test_scaled))[[target, ystar_col]]

In [ ]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

In [ ]:
for c in df_out.columns:
    df_out[c] = df_out[c] * target_stdev + target_mean

In [ ]:
df_out

,Close_lead15,Model forecast
DateTime,,
2021-01-03 18:14:00,1910.765743,1911.357544
2021-01-03 18:15:00,1911.158750,1911.268311
2021-01-03 18:16:00,1912.568774,1911.327515
2021-01-03 18:17:00,1912.845779,1911.248291
2021-01-03 18:18:00,1913.718794,1911.721924
...,...,...
2021-02-11 21:50:00,1820.004183,1827.918579
2021-02-11 21:51:00,1820.194186,1828.087036
2021-02-11 21:52:00,1820.254187,1828.223877


In [ ]:
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

fig = px.line(df_out, labels=dict(created_at="Date", value="XAUUSD"))
fig.add_vline(x=test_start, line_width=4, line_dash="dash")
fig.add_annotation(xref="paper", x=0.75, yref="paper", y=0.8, text="Test set start", showarrow=False)
fig.update_layout(
    template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()